The purpose of this notebook is to load the default cosplace model trained on sf_xs and evaluate recalls of various already trained re ranking models provided by [GeoWarp](https://github.com/gmberton/geo_warp)

# pip install requirements

Remember to click on "Restart Runtime" before go on

In [1]:
# CosPlace requirements
!pip3 install "faiss_cpu>=1.7.1"
!pip3 install "numpy>=1.21.2"
!pip3 install "Pillow>=9.0.1"
!pip3 install "scikit_learn>=1.0.2"
!pip3 install "torch>=1.8.2"
!pip3 install "torchvision>=0.9.2"
!pip3 install "tqdm>=4.62.3"
!pip3 install "utm>=0.7.0"
!pip3 install "kornia"

import torch
#use GPU if available 
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #'cpu' # 'cuda' or 'cpu'
print(DEVICE)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.0 MB 10.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.2 MB 28.3 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6108 sha256=a71d6909dd3d27d8d13655eda60342312a5e242fd36e489de4625608c262a69d
  Stored in directory: /root/.cache/pip/wheels/65/e2/d8/878a8cc986641056fbfebefc4d8eb64238a7b6d3426e86b447
Successfully built utm
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 35.6 MB/s 
cuda:0


# Download Datasets and previous data

Downloading with gdown doesn't work properly.

Prefer always to use drive / mount

In [1]:
import os
import gdown
from google.colab import drive

def download(id, output=None, quiet=True):
  gdown.download(
    f"https://drive.google.com/uc?export=download&confirm=pbef&id={id}",
    output=output,
    quiet=quiet
  )


use_mount = True
if use_mount:
  drive.mount('/content/drive')

# TOKYO-XS DATASET
if not os.path.isdir("/content/tokyo_xs"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/tokyo-xs.zip"
  else:
    id = "1fBCnap5BRh36474cVkjvjlC-yUTEb1n3"
    download(id, quiet=False)                           # download from our gdrive
    !jar xvf "/content/tokyo-xs.zip"                    # unzip
    !rm -r "/content/tokyo-xs.zip"                      # remove .zip file

if not os.path.isdir("/content/tokyo_xs"):
  raise FileNotFoundError(f"Can't download tokyo xs")

#TOKYO NIGHT DATASET
if not os.path.isdir("/content/tokyo-night"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/tokyo-night.zip"
  else:
    id = "1EZJY2r5565-iVk2oEbTns0xc4F_em4Y4"
    download(id, quiet=False)                           # download from our gdrive
    !jar xvf "/content/tokyo-night.zip"                    # unzip
    !rm -r "/content/tokyo-night.zip"

if not os.path.isdir("/content/tokyo-night"):
  raise FileNotFoundError(f"Can't download tokyo night")

# SAN FRANCISCO - XS DATASET
if not os.path.isdir("/content/small"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/sf-xs.zip"
  else:
    id = "1brIxBJmOgvuzFbI57f5LxnMxjccUu993"
    download(id, quiet=False)                           # download
    !jar xvf "/content/sf-xs.zip"                       # unzip
    !rm -r "/content/sf-xs.zip"                         # remove .zip file

if not os.path.isdir("/content/small"):
  raise FileNotFoundError(f"Can't download sfxs")

# CHECK YOUR DRIVE FOLDER!
# pretrained baselines for geowarp
if not os.path.isdir("/content/pretrained_baselines/"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/new_shared/geowarp/pretrained_baselines.zip"

if not os.path.isdir("/content/pretrained_baselines/"):
  raise FileNotFoundError(f"Can't download pretrained_baselines for geowarp")

# CHECK YOUR DRIVE FOLDER!
# trained hr for geowarp
if not os.path.isdir("/content/trained_homography_regressions/"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/new_shared/geowarp/trained_homography_regressions.zip"

if not os.path.isdir("/content/trained_homography_regressions/"):
  raise FileNotFoundError(f"Can't download trained_homography_regressions for geowarp")

# CHECK YOUR DRIVE FOLDER!
if not os.path.isfile("/content/saved_models/default.pth"):
  if use_mount:
    !mkdir "/content/saved_models"
    !cp "/content/drive/MyDrive/new_shared/project6/default.pth" "/content/saved_models/default.pth"

if not os.path.isfile("/content/saved_models/default.pth"):
  raise FileNotFoundError(f"Can't find model to resume, please provide one")

Streaming output truncated to the last 5000 lines.
 inflated: small/val/queries/@0548306.36@4179951.08@10@S@037.76561@-122.45154@PYNmOa_558a_w78qYq1IaA@@330@@@@200802@@.jpg
 inflated: small/val/queries/@0548258.48@4179554.69@10@S@037.76204@-122.45211@GGArRnpN7_AJXIx-El-9XQ@@270@@@@200711@@.jpg
 inflated: small/val/queries/@0548258.61@4173657.59@10@S@037.70889@-122.45250@5ktSqfqc-u2QOs68lw9sDw@@330@@@@201409@@.jpg
 inflated: small/val/queries/@0548302.51@4176700.15@10@S@037.73631@-122.45180@ey4sWjO5ilTkK9KTljV8dA@@60@@@@201903@@.jpg
 inflated: small/val/queries/@0548258.61@4173657.59@10@S@037.70889@-122.45250@5ktSqfqc-u2QOs68lw9sDw@@90@@@@201409@@.jpg
 inflated: small/val/queries/@0548302.51@4176700.15@10@S@037.73631@-122.45180@ey4sWjO5ilTkK9KTljV8dA@@330@@@@201903@@.jpg
 inflated: small/val/queries/@0548301.62@4177603.29@10@S@037.74445@-122.45175@6NkmeMiERhCVaKSI9mFdHQ@@0@@@@202102@@.jpg
 inflated: small/val/queries/@0548302.51@4176700.15@10@S@037.73631@-122.45180@ey4sWjO5ilTkK9KTljV8d

# Download Code

Clone of original repo of CosPlace and our code

In [1]:
# download code of CosPlace
!git clone "https://github.com/gmberton/CosPlace" 
#!rm -r "/content/CosPlace"

# download code of GeoWarp
!git clone "https://github.com/gmberton/geo_warp"

# download our code
#!rm -rf Team
!git clone --single-branch --branch "develop" "https://github.com/gab-palmeri/aml-geolocalization.git"
!mv "/content/aml-geolocalization/" "/content/Team"
#!rm -r "/content/aml-geolocalization"

fatal: destination path 'CosPlace' already exists and is not an empty directory.
fatal: destination path 'geo_warp' already exists and is not an empty directory.
Cloning into 'aml-geolocalization'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 161 (delta 26), reused 28 (delta 25), pack-reused 125
Receiving objects: 100% (161/161), 1.97 MiB | 11.08 MiB/s, done.
Resolving deltas: 100% (57/57), done.




# Import Code


In [10]:
import os
import sys
import torch
import logging
import multiprocessing
import numpy as np
import torchvision.transforms as T
from tqdm import tqdm
from datetime import datetime

sys.path.append("/content/CosPlace/")
sys.path.append("/content/geo_warp/")
sys.path.append("/content/Team/")
import CosPlace
from CosPlace import *

from Team import *

torch.backends.cudnn.benchmark = True  # Provides a speedup

This class let us to access to dictionary keys like `dict.key` instead of `dict["key"]`

In [11]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

Drive related functions to save and load training checkpoints

In [12]:
import os
from google.colab import drive

drive.mount('/content/drive')

def drive_save_checkpoint(output_folder, time):
    drive_folder = "/content/drive/MyDrive/project6/"
    if not os.path.exists(drive_folder):
        !mkdir "/content/drive/MyDrive/project6/"
    !zip -r "/content/drive/MyDrive/project6/{time}.zip" "/content/{output_folder}"

def drive_load_checkpoint(input_folder, time):
    drive_folder = "/content/drive/MyDrive/project6/"
    !jar xvf "/content/drive/MyDrive/project6/{time}.zip"

def drive_tester(output_folder):
    drive_folder = "/content/drive/MyDrive/project6/"
    if not os.path.exists(drive_folder):
        !mkdir "/content/drive/MyDrive/project6/"
    !touch "/content/drive/MyDrive/project6/test"

    if os.path.exists("/content/drive/MyDrive/project6/test"):
        logging.info("Drive saving works")
    else:
        logging.info("WARNING: Drive saving does not work")

# use this function when content on drive are not updated
def drive_refresh():
  drive.flush_and_unmount()
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Parameters

Original code provides two main executable files: `train.py` and `eval.py`. We want to reproduce the same behaviour of these two files so these are the parameters passed via terminal. They will be put in a dict called `args` to simply reuse code inside ex files

### Save dir for CosFace

In [13]:
saveDirLocal = "cosface"

## Setup parameters

In [14]:
# CosPlace Groups parameters
COS_M = 10
ALPHA = 30
COS_N = 5
COS_L = 2
GROUPS_NUM = 1
MIN_IMAGES_PER_CLASS = 10
# Model parameters
BACKBONE = "resnet18"   # ["resnet18", "efficientnet_v2_s", "mobilenet_v3_large"]
PRETRAIN = "imagenet"   # ["imagenet", "places", "gldv2"]
FC_OUTPUT_DIM = 512     # Output dimension of final fully connected layer
# Training parameters
AUGMENTATION_DEVICE = "cuda"    # ["cuda", "cpu"]
USE_AMP_16 = AUGMENTATION_DEVICE == "cuda"      # use Automatic Mixed Precision
BATCH_SIZE = 32
EPOCHS_NUM = 3
ITERATIONS_PER_EPOCH = 10_000
LR = 0.00001                    # Learning rate  
CLASSIFIERS_LR = 0.01
# Data augmentation
BRIGHTNESS = 0.7
CONTRAST = 0.7
SATURATION = 0.7
HUE = 0.5
RANDOM_RESIZED_CROP = 0.5
# Validation / test parameters
INFER_BATCH_SIZE = 16           # Batch size for inference (validating and testing)
POSITIVE_DIST_THRESHOLD = 25    # distance in meters for a prediction to be considered a positive
# Resume parameters
RESUME_TRAIN = None     # path to checkpoint to resume, e.g. logs/.../last_checkpoint.pth
RESUME_MODEL = "/content/saved_models/default.pth"     # Path to model to resume for test
# Other parameters
DEVICE = "cuda"                     # ["cuda", "cpu"]
SEED = 0
NUM_WORKERS = 8
DATASET_FOLDER = "/content/small"   # path of the folder with train/val sets
SAVEDIR = saveDirLocal


if not os.path.exists(DATASET_FOLDER):
    raise FileNotFoundError(f"Dataset folder {DATASET_FOLDER} not found")

train_set_folder = os.path.join(DATASET_FOLDER, "train")

if not os.path.exists(train_set_folder):
    raise FileNotFoundError(f"Train set folder {train_set_folder} not found")

val_set_folder = os.path.join(DATASET_FOLDER, "val")

if not os.path.exists(val_set_folder):
    raise FileNotFoundError(f"Validation set folder {val_set_folder} not found")


# dictionary for the parameters
args = {
    'M': COS_M, 'alpha': ALPHA, 'N': COS_N, 'L': COS_L, 'groups_num': GROUPS_NUM,
    'min_images_per_class': MIN_IMAGES_PER_CLASS, 'backbone': BACKBONE, "pretrain": PRETRAIN,
    'fc_output_dim': FC_OUTPUT_DIM, 'use_amp16': USE_AMP_16,
    'augmentation_device': AUGMENTATION_DEVICE, 'batch_size': BATCH_SIZE,
    'epochs_num': EPOCHS_NUM, 'iterations_per_epoch': ITERATIONS_PER_EPOCH,
    'lr': LR, 'classifiers_lr': CLASSIFIERS_LR, 'brightness': BRIGHTNESS,
    'contrast': CONTRAST, 'hue': HUE, 'saturation': SATURATION,
    'random_resized_crop': RANDOM_RESIZED_CROP, 'infer_batch_size': INFER_BATCH_SIZE,
    'positive_dist_threshold': POSITIVE_DIST_THRESHOLD, 'resume_train': RESUME_TRAIN,
    'resume_model': RESUME_MODEL, 'device': DEVICE, 'seed': SEED,
    'num_workers': NUM_WORKERS, 'dataset_folder': DATASET_FOLDER, 'save_dir': SAVEDIR,
    'val_set_folder': val_set_folder, 'train_set_folder': train_set_folder,
}

# this helps to reuse the code from the original CosPlace
args = dotdict(args)


# Setup logging

In [15]:
from CosPlace import commons

start_time = datetime.now()
output_folder = f"logs/{args.save_dir}/{start_time.strftime('%Y-%m-%d_%H-%M-%S')}"
commons.make_deterministic(args.seed)
commons.setup_logging(output_folder, console=None)
logging.info(" ".join(sys.argv))
logging.info(f"Arguments: {args}")
logging.info(f"The outputs are being saved in {output_folder}")

INFO:root:/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-529fc7f6-9be3-45cc-933e-0a99a9d97ce2.json
INFO:root:Arguments: {'M': 10, 'alpha': 30, 'N': 5, 'L': 2, 'groups_num': 1, 'min_images_per_class': 10, 'backbone': 'resnet18', 'fc_output_dim': 512, 'use_amp_16': 'store_true', 'augmentation_device': 'cuda', 'batch_size': 32, 'epochs_num': 3, 'iterations_per_epoch': 10000, 'lr': 1e-05, 'classifiers_lr': 0.01, 'brightness': 0.7, 'contrast': 0.7, 'hue': 0.5, 'saturation': 0.7, 'random_resized_crop': 0.5, 'infer_batch_size': 16, 'positive_dist_threshold': 25, 'resume_train': None, 'resume_model': '/content/saved_models/default.pth', 'device': 'cuda', 'seed': 0, 'num_workers': 8, 'dataset_folder': '/content/small', 'save_dir': 'cosface', 'val_set_folder': '/content/small/val', 'train_set_folder': '/content/small/train'}
INFO:root:The outputs are being saved in logs/cosface/2022-12-28_15-26-42


# Test

## Import model

In [16]:
from CosPlace import model, test, datasets
from Team.model import network
from datasets.test_dataset import TestDataset

#### Model
model = network.GeoLocalizationNet(args.backbone, args.fc_output_dim)

logging.info(f"There are {torch.cuda.device_count()} GPUs and {multiprocessing.cpu_count()} CPUs.")

if args.resume_model is not None:
  if os.path.exists(args.resume_model):
    resume_model = args.resume_model

if resume_model is not None:
  if not os.path.exists(resume_model):
    raise FileNotFoundError(f"Model {resume_model} not found.")

  logging.info(f"Loading model from {resume_model}")
  model_state_dict = torch.load(resume_model)
  model.load_state_dict(model_state_dict)
else:
    logging.info("WARNING: You didn't provide a path to resume the model (--resume_model parameter). " +
                 "Evaluation will be computed using randomly initialized weights.")

model = model.to(args.device)

DEBUG:root:Train only layer3 and layer4 of the resnet18, freeze the previous ones
INFO:root:There are 1 GPUs and 2 CPUs.
INFO:root:Loading model from /content/saved_models/default.pth


## GeoWarp Settings

In [17]:
import itertools 
# architectures for which we have pretrained feature extractor
archs = ["alexnet", "resnet50", "vgg16"]
# pooling layer
poolings = ["gem", "netvlad"]
# num_reranked_predictions
nums_reranked = [5, 10, 15]

combinations = itertools.product(archs, poolings, nums_reranked)


## function to calculate predictions and recalls for GeoLocalizationNet

In [18]:
import faiss
import torch
import logging
from PIL import Image
import numpy as np
from tqdm import tqdm
from typing import Tuple
from argparse import Namespace
from torch.utils.data.dataset import Subset
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import Team.GeoWarp.network as network
import Team.GeoWarp.util as util
import Team.GeoWarp.dataset_warp as dataset_warp

def calculate_preds_and_recalls(args: Namespace, eval_ds: Dataset, model: torch.nn.Module):
    """Compute descriptors of the given dataset and compute the recalls."""
    
    model = model.eval()
    with torch.no_grad():
        logging.debug("Extracting database descriptors for evaluation/testing")
        database_subset_ds = Subset(eval_ds, list(range(eval_ds.database_num)))
        database_dataloader = DataLoader(dataset=database_subset_ds, num_workers=args.num_workers,
                                         batch_size=args.infer_batch_size, pin_memory=(args.device == "cuda"))
        all_descriptors = np.empty((len(eval_ds), args.fc_output_dim), dtype="float32")
        for images, indices in tqdm(database_dataloader, ncols=100):
            descriptors = model(images.to(args.device))
            descriptors = descriptors.cpu().numpy()
            all_descriptors[indices.numpy(), :] = descriptors
        
        logging.debug("Extracting queries descriptors for evaluation/testing using batch size 1")
        queries_infer_batch_size = 1
        queries_subset_ds = Subset(eval_ds, list(range(eval_ds.database_num, eval_ds.database_num+eval_ds.queries_num)))
        queries_dataloader = DataLoader(dataset=queries_subset_ds, num_workers=args.num_workers,
                                        batch_size=queries_infer_batch_size, pin_memory=(args.device == "cuda"))
        for images, indices in tqdm(queries_dataloader, ncols=100):
            descriptors = model(images.to(args.device))
            descriptors = descriptors.cpu().numpy()
            all_descriptors[indices.numpy(), :] = descriptors
    
    queries_descriptors = all_descriptors[eval_ds.database_num:]
    database_descriptors = all_descriptors[:eval_ds.database_num]
    
    # Use a kNN to find predictions
    faiss_index = faiss.IndexFlatL2(args.fc_output_dim)
    faiss_index.add(database_descriptors)
    del database_descriptors, all_descriptors
    
    logging.debug("Calculating recalls")
    _, predictions = faiss_index.search(queries_descriptors, max(RECALL_VALUES))
    
    #### For each query, check if the predictions are correct
    positives_per_query = eval_ds.get_positives()
    recalls = np.zeros(len(RECALL_VALUES))
    for query_index, preds in enumerate(predictions):
        for i, n in enumerate(RECALL_VALUES):
            if np.any(np.in1d(preds[:n], positives_per_query[query_index])):
                recalls[i:] += 1
                break
    # Divide by queries_num and multiply by 100, so the recalls are in percentages
    recalls = recalls / eval_ds.queries_num * 100
    recalls_str = ", ".join([f"R@{val}: {rec:.1f}" for val, rec in zip(RECALL_VALUES, recalls)])
    return predictions, recalls, recalls_str


## geo warp test function
starting from predictions, arch, pooling and paths (all args) 

In [19]:


# Compute R@1, R@5, R@10, R@20
RECALL_VALUES = [1, 5, 10, 20]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def open_image_and_apply_transform(image_path):
    """Given the path of an image, open the image, and return it as a normalized tensor.
    """
    
    pil_image = Image.open(image_path)
    tensor_image = transform(pil_image)
    return tensor_image


def geo_warp_test(warp_model, predictions, test_dataset, num_reranked_predictions=5, recall_values=[1, 5, 10, 20], test_batch_size=8):
    
    warp_model.eval()
    reranked_predictions = predictions.copy()

    recalls = None
    recalls_pretty_str = None

    with torch.no_grad():
      for num_q in tqdm(range(test_dataset.queries_num), desc="Testing", ncols=100):
              dot_prods_wqp = np.zeros((num_reranked_predictions))
              query_path = test_dataset.queries_paths[num_q]
              for i1 in range(0, num_reranked_predictions, test_batch_size):
                  batch_indexes = list(range(num_reranked_predictions))[i1:i1+test_batch_size]
                  current_batch_size = len(batch_indexes)
                  query = open_image_and_apply_transform(query_path)
                  query_repeated_twice = torch.repeat_interleave(query.unsqueeze(0), current_batch_size, 0)
                  
                  preds = []
                  for i in batch_indexes:
                      pred_path = test_dataset.database_paths[predictions[num_q, i]]
                      preds.append(open_image_and_apply_transform(pred_path))
                  preds = torch.stack(preds)
                  
                  warped_pair = dataset_warp.compute_warping(warp_model, query_repeated_twice.cuda(), preds.cuda())
                  q_features = warp_model("features_extractor", [warped_pair[0], "local"])
                  p_features = warp_model("features_extractor", [warped_pair[1], "local"])
                  # Sum along all axes except for B. wqp stands for warped query-prediction
                  dot_prod_wqp = (q_features * p_features).sum(list(range(1, len(p_features.shape)))).cpu().detach().numpy()
                  
                  dot_prods_wqp[i1:i1+test_batch_size] = dot_prod_wqp
              
              reranking_indexes = dot_prods_wqp.argsort()[::-1]
              reranked_predictions[num_q, :num_reranked_predictions] = predictions[num_q][reranking_indexes]
      
      ground_truths = test_dataset.get_positives()
      recalls, recalls_pretty_str = util.compute_recalls(reranked_predictions, ground_truths, test_dataset, RECALL_VALUES)
    return recalls, recalls_pretty_str



## Test on SF-XS

### Calculate predictions for SF-XS

In [23]:
# dataset_folder is the same of the training
test_set_folder = os.path.join(DATASET_FOLDER, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder, queries_folder="queries_v1",
                      positive_dist_threshold=args.positive_dist_threshold)

sf_xs_preds, recalls, recalls_str = calculate_preds_and_recalls(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
sf_xs_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

DEBUG:root:Extracting database descriptors for evaluation/testing
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|███████████████████████████████████████████████████████████| 1700/1700 [03:36<00:00,  7.85it/s]
DEBUG:root:Extracting queries descriptors for evaluation/testing using batch size 1
100%|███████████████████████████████████████████████████████████| 1000/1000 [00:16<00:00, 60.93it/s]
DEBUG:root:Calculating recalls
INFO:root:< test - #q: 1000; #db: 27191 >: R@1: 52.2, R@5: 66.3, R@10: 71.8, R@20: 76.3


Benchmark geowarp

In [24]:
combinations = itertools.product(archs, poolings, nums_reranked)
sf_xs_rerank = []

for arch, pooling, num_rerank in combinations:
  if arch == "vgg16" and not num_rerank==5: break
  ## MODEL
  features_extractor = network.FeaturesExtractor(arch, pooling)
  #global_features_dim = commons_warp.get_output_dim(features_extractor, "GEM")
                
  state_dict = torch.load(f"/content/pretrained_baselines/{arch}_{pooling}.pth")
  features_extractor.load_state_dict(state_dict)
  del state_dict
                
  state_dict = torch.load(f"/content/trained_homography_regressions/{arch}_{pooling}.pth")
  homography_regression = network.HomographyRegression(kernel_sizes=[7, 5, 5, 5, 5, 5], channels=[225, 128, 128, 64, 64, 64, 64], padding=1)
  homography_regression.load_state_dict(state_dict)
  del state_dict
                
  warp_model = network.Network(features_extractor, homography_regression).cuda().eval()
  warp_model = torch.nn.DataParallel(warp_model)
  
  batch_size = 8
  recalls, recalls_str = geo_warp_test(warp_model=warp_model, predictions=sf_xs_preds, test_dataset=test_ds, num_reranked_predictions=num_rerank, test_batch_size = batch_size)
  logging.info(f"arch: {arch}, pooling: {pooling}, num_rerank: {num_rerank}")
  logging.info(f"{test_ds}: {recalls_str}")
  sf_xs_rerank.append(recalls)

  del warp_model
  del features_extractor
  del homography_regression
del sf_xs_preds

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Testing: 100%|██████████████████████████████████████████████████| 1000/1000 [01:41<00:00,  9.86it/s]
INFO:root:arch: alexnet, pooling: netvlad, num_rerank: 5
INFO:root:< test - #q: 1000; #db: 27191 >: R@1: 53.6, R@5: 66.3, R@10: 71.8, R@20: 76.3
Testing: 100%|██████████████████████████████████████████████████| 1000/1000 [03:27<00:00,  4.81it/s]
INFO:root:arch: alexnet, pooling: 

## Test on Tokyo-XS

### Calculate predictions for Tokyo XS

In [25]:
tokyo_xs_folder = "/content/tokyo_xs"
test_set_folder = os.path.join(tokyo_xs_folder, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder,
                      positive_dist_threshold=args.positive_dist_threshold)

tokyo_xs_preds, recalls, recalls_str = calculate_preds_and_recalls(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
tokyo_xs_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

DEBUG:root:Extracting database descriptors for evaluation/testing
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|█████████████████████████████████████████████████████████████| 799/799 [01:59<00:00,  6.69it/s]
DEBUG:root:Extracting queries descriptors for evaluation/testing using batch size 1
100%|█████████████████████████████████████████████████████████████| 315/315 [00:05<00:00, 55.31it/s]
DEBUG:root:Calculating recalls
INFO:root:< test - #q: 315; #db: 12771 >: R@1: 69.5, R@5: 84.8, R@10: 89.2, R@20: 93.0


Benchmark geowarp

In [26]:
combinations = itertools.product(archs, poolings, nums_reranked)
tokyo_xs_rerank = []
for arch, pooling, num_rerank in combinations:
  if arch == "vgg16" and not num_rerank==5: break
  ## MODEL
  features_extractor = network.FeaturesExtractor(arch, pooling)
  #global_features_dim = commons_warp.get_output_dim(features_extractor, "GEM")
                
  state_dict = torch.load(f"/content/pretrained_baselines/{arch}_{pooling}.pth")
  features_extractor.load_state_dict(state_dict)
  del state_dict
                
  state_dict = torch.load(f"/content/trained_homography_regressions/{arch}_{pooling}.pth")
  homography_regression = network.HomographyRegression(kernel_sizes=[7, 5, 5, 5, 5, 5], channels=[225, 128, 128, 64, 64, 64, 64], padding=1)
  homography_regression.load_state_dict(state_dict)
  del state_dict
                
  warp_model = network.Network(features_extractor, homography_regression).cuda().eval()
  warp_model = torch.nn.DataParallel(warp_model)

  recalls, recalls_str = geo_warp_test(warp_model=warp_model, predictions=tokyo_xs_preds, test_dataset=test_ds, num_reranked_predictions=num_rerank)
  logging.info(f"arch: {arch}, pooling: {pooling}, num_rerank: {num_rerank}")
  logging.info(f"{test_ds}: {recalls_str}")
  tokyo_xs_rerank.append(recalls)

  del warp_model
  del features_extractor
  del homography_regression
del tokyo_xs_preds

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Testing: 100%|████████████████████████████████████████████████████| 315/315 [00:34<00:00,  9.22it/s]
INFO:root:arch: alexnet, pooling: netvlad, num_rerank: 5
INFO:root:< test - #q: 315; #db: 12771 >: R@1: 75.2, R@5: 84.8, R@10: 89.2, R@20: 93.0
Testing: 100%|████████████████████████████████████████████████████| 315/315 [01:09<00:00,  4.56it/s]
INFO:root:arch: alexnet, pooling: n

## Test on Tokyo-Night

### Calculate predictions for Tokyo Night

In [21]:
tokyo_night_folder = "/content/tokyo-night/"
test_set_folder = os.path.join(tokyo_night_folder, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder,
                      positive_dist_threshold=args.positive_dist_threshold)

tokyo_night_preds, recalls, recalls_str = calculate_preds_and_recalls(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
tokyo_night_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

DEBUG:root:Extracting database descriptors for evaluation/testing
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|█████████████████████████████████████████████████████████████| 799/799 [02:02<00:00,  6.54it/s]
DEBUG:root:Extracting queries descriptors for evaluation/testing using batch size 1
100%|█████████████████████████████████████████████████████████████| 105/105 [00:02<00:00, 45.17it/s]
DEBUG:root:Calculating recalls
INFO:root:< test - #q: 105; #db: 12771 >: R@1: 50.5, R@5: 72.4, R@10: 79.0, R@20: 85.7


Benchmark geowarp

In [22]:
combinations = itertools.product(archs, poolings, nums_reranked)
tokyo_night_rerank = []
for arch, pooling, num_rerank in combinations:
  if arch == "vgg16" and not num_rerank==5: break
  ## MODEL
  features_extractor = network.FeaturesExtractor(arch, pooling)
  #global_features_dim = commons_warp.get_output_dim(features_extractor, "GEM")
                
  state_dict = torch.load(f"/content/pretrained_baselines/{arch}_{pooling}.pth")
  features_extractor.load_state_dict(state_dict)
  del state_dict
                
  state_dict = torch.load(f"/content/trained_homography_regressions/{arch}_{pooling}.pth")
  homography_regression = network.HomographyRegression(kernel_sizes=[7, 5, 5, 5, 5, 5], channels=[225, 128, 128, 64, 64, 64, 64], padding=1)
  homography_regression.load_state_dict(state_dict)
  del state_dict
                
  warp_model = network.Network(features_extractor, homography_regression).cuda().eval()
  warp_model = torch.nn.DataParallel(warp_model)

  recalls, recalls_str = geo_warp_test(warp_model=warp_model, predictions=tokyo_night_preds, test_dataset=test_ds, num_reranked_predictions=num_rerank)
  logging.info(f"arch: {arch}, pooling: {pooling}, num_rerank: {num_rerank}")
  logging.info(f"{test_ds}: {recalls_str}")
  tokyo_night_rerank.append(recalls)

  del warp_model
  del features_extractor
  del homography_regression
del tokyo_night_preds

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Testing: 100%|████████████████████████████████████████████████████| 105/105 [00:11<00:00,  8.77it/s]
INFO:root:arch: alexnet, pooling: netvlad, num_rerank: 5
INFO:root:< test - #q: 105; #db: 12771 >: R@1: 61.9, R@5: 72.4, R@10: 79.0, R@20: 85.7
Testing: 100%|████████████████████████████████████████████████████| 105/105 [00:24<00:00,  4.37it/s]
INFO:root:arch: alexnet, pooling: n

# Results

## San Francisco XS

In [ ]:
logging.info("Original sfxs R1/R5")
logging.info(sf_xs_r15)

combinations = itertools.product(archs, poolings, nums_reranked)
for arch, pooling, num in combinations:
  try:
    logging.info(f"Recalls sfxs for arch: {arch}, pooling: {pooling}, num_rerank: {num}")
  except:
    break
  logging.info(sf_xs_rerank)


## Tokyo XS

In [20]:
logging.info("Original tokyoxs R1/R5")
logging.info(tokyo_xs_r15)

combinations = itertools.product(archs, poolings, nums_reranked)
for arch, pooling, num in combinations:
  try:
    logging.info(f"Recalls tokyoxs for arch: {arch}, pooling: {pooling}, num_rerank: {num}")
  except:
    break
  logging.info(tokyo_xs_rerank)

INFO:root:Original tokyoxs R1/R5
INFO:root:69.5/84.8
INFO:root:Recalls tokyoxs for arch: alexnet, pooling: gem, num_rerank: 5
INFO:root:[array([77.46031746, 84.76190476, 89.20634921, 93.01587302]), array([76.50793651, 87.3015873 , 89.20634921, 93.01587302]), array([74.92063492, 87.3015873 , 91.11111111, 93.01587302]), array([76.19047619, 84.76190476, 89.20634921, 93.01587302]), array([76.82539683, 86.66666667, 89.20634921, 93.01587302]), array([75.87301587, 86.98412698, 91.42857143, 93.01587302]), array([79.36507937, 84.76190476, 89.20634921, 93.01587302])]
INFO:root:Recalls tokyoxs for arch: alexnet, pooling: gem, num_rerank: 10
INFO:root:[array([77.46031746, 84.76190476, 89.20634921, 93.01587302]), array([76.50793651, 87.3015873 , 89.20634921, 93.01587302]), array([74.92063492, 87.3015873 , 91.11111111, 93.01587302]), array([76.19047619, 84.76190476, 89.20634921, 93.01587302]), array([76.82539683, 86.66666667, 89.20634921, 93.01587302]), array([75.87301587, 86.98412698, 91.42857143, 

## Tokyo Night

In [21]:
logging.info("Original tokyo night R1/R5")
logging.info(tokyo_night_r15)

combinations = itertools.product(archs, poolings, nums_reranked)
for arch, pooling, num in combinations:
  try:
    logging.info(f"Recalls tokyo night for arch: {arch}, pooling: {pooling}, num_rerank: {num}")
  except:
    break
  logging.info(tokyo_night_rerank)

INFO:root:Original tokyo night R1/R5
INFO:root:50.5/72.4
INFO:root:Recalls tokyo night for arch: alexnet, pooling: gem, num_rerank: 5
INFO:root:[array([60.95238095, 72.38095238, 79.04761905, 85.71428571]), array([57.14285714, 75.23809524, 79.04761905, 85.71428571]), array([53.33333333, 75.23809524, 81.9047619 , 85.71428571]), array([62.85714286, 72.38095238, 79.04761905, 85.71428571]), array([62.85714286, 76.19047619, 79.04761905, 85.71428571]), array([61.9047619 , 77.14285714, 83.80952381, 85.71428571]), array([66.66666667, 72.38095238, 79.04761905, 85.71428571])]
INFO:root:Recalls tokyo night for arch: alexnet, pooling: gem, num_rerank: 10
INFO:root:[array([60.95238095, 72.38095238, 79.04761905, 85.71428571]), array([57.14285714, 75.23809524, 79.04761905, 85.71428571]), array([53.33333333, 75.23809524, 81.9047619 , 85.71428571]), array([62.85714286, 72.38095238, 79.04761905, 85.71428571]), array([62.85714286, 76.19047619, 79.04761905, 85.71428571]), array([61.9047619 , 77.14285714, 8

# Save logs

In [27]:
import os
from google.colab import drive

drive.mount('/content/drive')

# zip logs -> logs.zip
!zip -r /content/drive/MyDrive/new_shared/geowarp/logs.zip /content/logs/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  adding: content/logs/ (stored 0%)
  adding: content/logs/cosface/ (stored 0%)
  adding: content/logs/cosface/2022-12-28_15-20-26/ (stored 0%)
  adding: content/logs/cosface/2022-12-28_15-20-26/info.log (deflated 50%)
  adding: content/logs/cosface/2022-12-28_15-20-26/debug.log (deflated 52%)
  adding: content/logs/cosface/2022-12-28_14-28-30/ (stored 0%)
  adding: content/logs/cosface/2022-12-28_14-28-30/info.log (deflated 93%)
  adding: content/logs/cosface/2022-12-28_14-28-30/debug.log (deflated 93%)
  adding: content/logs/cosface/2022-12-28_15-25-50/ (stored 0%)
  adding: content/logs/cosface/2022-12-28_15-25-50/info.log (deflated 80%)
  adding: content/logs/cosface/2022-12-28_15-25-50/debug.log (deflated 81%)
  adding: content/logs/cosface/2022-12-28_15-26-42/ (stored 0%)
  adding: content/logs/cosface/2022-12-28_15-26-42/info.log (deflated 77%)
  addin